In [1]:
%load_ext autoreload
%autoreload 2
import gc
import re
import pandas as pd
from __future__ import print_function, division
import numpy as np
pd.set_option('max_columns', 200)
pd.set_option('max_rows', 200)
import os
import sys
HOME = os.path.expanduser("~")
sys.path.append(f'{HOME}/kaggle/data_analysis/library')
import utils
from utils import get_categorical_features, get_numeric_features, reduce_mem_usage, elo_save_feature
from preprocessing import get_dummies
import datetime

from tqdm import tqdm
import time
import sys
import glob
from joblib import Parallel, delayed

logger = utils.logger_func()

key = 'card_id'
target = 'target'
ignore_list = [key, target, 'merchant_id', 'first_avtive_month']
start_time = "{0:%Y%m%d_%H%M%S}".format(datetime.datetime.now())

2019-02-25 15:32:24,143 utils 400 [INFO]    [logger_func] start 


In [3]:
is_binary = 1
fname = 'auth1'
df = utils.read_df_pkl('../input/auht1_fu*').reset_index()
# df = utils.read_df_pkl('../input/auht0_future_amount_pred0*').reset_index()
target = 'target'
value = 'purchase_amount_new_sum'
df[target] = df.groupby(key)[value].shift(-1)
df[target] = df[target].replace(-1, -10)

# binary version
if is_binary:
    df[target] = df[target].map(lambda x: 0 if x<0.5 else 1)
df.head()

100%|██████████| 3/3 [00:02<00:00,  1.03it/s]


,card_id,month_lag,purchase_amount_new_sum,purchase_amount_new_mean,purchase_amount_new_max,purchase_amount_new_min,purchase_amount_new_std,authorized_flag_sum,authorized_flag_mean,installments_mean,installments_max,installments_min,yyyymmdd_nunique,purchase_date_count,subsector_id_nunique,merchant_category_id_nunique,merchant_id_nunique,state_id_nunique,city_id_nunique,category_1_sum,category_1_mean,auht1_roll_mean_window12_authorized_flag_mean,auht1_roll_mean_window12_authorized_flag_sum,auht1_roll_mean_window12_category_1_mean,auht1_roll_mean_window12_category_1_sum,auht1_roll_mean_window12_city_id_nunique,auht1_roll_mean_window12_installments_max,auht1_roll_mean_window12_installments_mean,auht1_roll_mean_window12_installments_min,auht1_roll_mean_window12_merchant_category_id_nunique,auht1_roll_mean_window12_merchant_id_nunique,auht1_roll_mean_window12_purchase_amount_new_max,auht1_roll_mean_window12_purchase_amount_new_mean,auht1_roll_mean_window12_purchase_amount_new_min,auht1_roll_mean_window12_purchase_amount_new_std,auht1_roll_mean_window12_purchase_amount_new_sum,auht1_roll_mean_window12_purchase_date_count,auht1_roll_mean_window12_state_id_nunique,auht1_roll_mean_window12_subsector_id_nunique,auht1_roll_mean_window12_yyyymmdd_nunique,auht1_roll_mean_window6_authorized_flag_mean,auht1_roll_mean_window6_authorized_flag_sum,auht1_roll_mean_window6_category_1_mean,auht1_roll_mean_window6_category_1_sum,auht1_roll_mean_window6_city_id_nunique,auht1_roll_mean_window6_installments_max,auht1_roll_mean_window6_installments_mean,auht1_roll_mean_window6_installments_min,auht1_roll_mean_window6_merchant_category_id_nunique,auht1_roll_mean_window6_merchant_id_nunique,auht1_roll_mean_window6_purchase_amount_new_max,auht1_roll_mean_window6_purchase_amount_new_mean,auht1_roll_mean_window6_purchase_amount_new_min,auht1_roll_mean_window6_purchase_amount_new_std,auht1_roll_mean_window6_purchase_amount_new_sum,auht1_roll_mean_window6_purchase_date_count,auht1_roll_mean_window6_state_id_nunique,auht1_roll_mean_window6_subsector_id_nunique,auht1_roll_mean_window6_yyyymmdd_nunique,auht1_roll_mean_window4_authorized_flag_mean,auht1_roll_mean_window4_authorized_flag_sum,auht1_roll_mean_window4_category_1_mean,auht1_roll_mean_window4_category_1_sum,auht1_roll_mean_window4_city_id_nunique,auht1_roll_mean_window4_installments_max,auht1_roll_mean_window4_installments_mean,auht1_roll_mean_window4_installments_min,auht1_roll_mean_window4_merchant_category_id_nunique,auht1_roll_mean_window4_merchant_id_nunique,auht1_roll_mean_window4_purchase_amount_new_max,auht1_roll_mean_window4_purchase_amount_new_mean,auht1_roll_mean_window4_purchase_amount_new_min,auht1_roll_mean_window4_purchase_amount_new_std,auht1_roll_mean_window4_purchase_amount_new_sum,auht1_roll_mean_window4_purchase_date_count,auht1_roll_mean_window4_state_id_nunique,auht1_roll_mean_window4_subsector_id_nunique,auht1_roll_mean_window4_yyyymmdd_nunique,auht1_roll_mean_window3_authorized_flag_mean,auht1_roll_mean_window3_authorized_flag_sum,auht1_roll_mean_window3_category_1_mean,auht1_roll_mean_window3_category_1_sum,auht1_roll_mean_window3_city_id_nunique,auht1_roll_mean_window3_installments_max,auht1_roll_mean_window3_installments_mean,auht1_roll_mean_window3_installments_min,auht1_roll_mean_window3_merchant_category_id_nunique,auht1_roll_mean_window3_merchant_id_nunique,auht1_roll_mean_window3_purchase_amount_new_max,auht1_roll_mean_window3_purchase_amount_new_mean,auht1_roll_mean_window3_purchase_amount_new_min,auht1_roll_mean_window3_purchase_amount_new_std,auht1_roll_mean_window3_purchase_amount_new_sum,auht1_roll_mean_window3_purchase_date_count,auht1_roll_mean_window3_state_id_nunique,auht1_roll_mean_window3_subsector_id_nunique,auht1_roll_mean_window3_yyyymmdd_nunique,auht1_roll_mean_window2_authorized_flag_mean,auht1_roll_mean_window2_authorized_flag_sum,auht1_roll_mean_window2_category_1_mean,auht1_roll_mean_window2_category_1_sum,auht1_roll_mean_window2_city_id_nunique,auht1

In [7]:
# import lightgbm as lgb
# from sklearn.metrics import mean_squared_error
# HOME = os.path.expanduser('~')
# sys.path.append(f'{HOME}/kaggle/data_analysis/model')
# from params_lgbm import params_elo

# score_list = []
# is_weight = 1

# params = params_elo()[1]
# if is_binary:
#     metric = 'auc'
# else:
#     metric = 'rmse'
    
# params['metric'] = metric
# num_leaves = 16
# params['learning_rate'] = 0.01
# params['num_leaves'] = num_leaves
# params['subsample'] = 0.9
# params['colsample_bytree'] = 0.9
# # params['colsample_bytree'] = 0.3
# params['min_child_samples'] = 20
# params['lambda_l2'] = 1

# valid_list = [-2, -3, -4]
# ignore_list = [key, target, 'merchant_id', 'first_active_month', 'index', 'personal_term', 'clf_pred', 'group']
# use_cols = [col for col in df.columns if col not in ignore_list]
# use_cols = [col for col in df.columns if col not in ignore_list and not(col.count('roll'))]
# test = df[df['month_lag']  == -1]
# y_test = np.zeros(len(test))
# # oof = np.zeros(len(df))

# for valid_month in valid_list:
#     train = df[df['month_lag'] < valid_month]
#     valid = df[df['month_lag'] ==valid_month]
    
#     x_train, y_train = train[use_cols], train[target].values
#     x_val, y_val = valid[use_cols], valid[target].values
    
#     lgb_train = lgb.Dataset(x_train, y_train)
#     lgb_eval = lgb.Dataset(x_val, y_val)
    
#     LGBM = lgb.train(
#         params=params
#         ,train_set=lgb_train
#         ,valid_sets=lgb_eval
#         ,verbose_eval=100
#         ,early_stopping_rounds=150
#         ,num_boost_round=20000
#         ,categorical_feature=[]
#     )
# #     oof = df[df['month_lag'] >=valid_month][use_cols]
# #     oof_pred = LGBM.predict(oof)
    
#     if is_weight:
# #         if valid_month==-1:
# #             y_test += LGBM.predict(test[use_cols])*0.6
#         if valid_month==-2:
#             y_test += LGBM.predict(test[use_cols])*0.6
#         elif valid_month==-3:
#             y_test += LGBM.predict(test[use_cols])*0.3
#         elif valid_month==-4:
#             y_test += LGBM.predict(test[use_cols])*0.1
#         y_pred = LGBM.predict(x_val)
#         score = np.sqrt(mean_squared_error(y_val, y_pred))
#         score_list.append(score)
#         print(f"Valid:{valid_month} | Score: {score}")
#         print(y_test[:5])
# #     else:
# #         y_test += LGBM.predict(test[use_cols])
        
# # if not(is_weight):
# y_test = y_test / len(valid_list)
        
# cv_score = np.mean(score_list)
# logger.info(f"CV: {cv_score}")

# test['prediction'] = y_test
if not(is_binary):
    test['prediction_diff_lastmonth'] = y_test - test['purchase_amount_new_sum'].values
    test['prediction_ratio_lastmonth'] = y_test / test['purchase_amount_new_sum'].values
    pred_cols = [col for col in test.columns if col.count('pred')]
else:
    test['prediction_diff_lastmonth'] = y_test - test[target].values
    pred_cols = ['prediction_diff_lastmonth']
df_pred = test[[key] + pred_cols]

base = utils.read_pkl_gzip('../input/base_type_group.gz')[[key, target]].set_index(key)
df_pred.set_index(key, inplace=True)
base = base.join(df_pred)
for col in pred_cols:
    feature = base[col].values
    if is_binary:
        utils.to_pkl_gzip(obj=feature, path=f"../features/1_first_valid/265_clf_feature_future_amount_{fname}_{col}_RMSE_CV{str(cv_score).replace('.', '-')}")
    else:
        utils.to_pkl_gzip(obj=feature, path=f"../features/1_first_valid/265_prd_feature_future_amount_{fname}_{col}_RMSE_CV{str(cv_score).replace('.', '-')}")

In [12]:
feature.shape

(325540,)